<a href="https://colab.research.google.com/github/dmuinoo/gestual/blob/main/YoloX_Custom_Training_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **YoloX Custom Training - Emotion Detection**


![image:png](https://github.com/Megvii-BaseDetection/YOLOX/raw/main/assets/logo.png)


<p>YoloX just got released, and it is better and faster than, YoloR, YoloV4, Scaled YoloV4, YoloV5 and PP-YOLOv2.</p>


<p>In this Tutorial let us learn how to do Emotion Detection by doing custom training on YoloX. All right here and right now. No time to waste guys, lets get started!!</p>



# **Want to Become a YOLOX Expert?**
💻Get Started with YOLOX [Get Started](https://www.augmentedstartups.com/yolox-registration). <br>
⭐ Download the Code at the [AI Vision Store](https://augmentedstartups.info/VisionStore)<br>
☕ Buy me [Chai/Coffee](https://bit.ly/BuymeaCoffeeAS)

# **About US**
[Augmented Startups](https://www.augmentedstartups.com) provides tutorials in AI Computer Vision and Augmented Reality. With over **95K subscribers** on our channel, we teach state-of-art models and build apps and projects that solve real-world problems. 

![image:png](https://drive.google.com/uc?export=view&id=1ps6pORV6b4eOoQAljv8s-hTq9YAaIN3b)

<br>

#From the Author   
## hey there <img src="https://media.giphy.com/media/hvRJCLFzcasrR4ia7z/giphy.gif" width="25px">

**I am Rohit Kukreja - A Computer Vision & Deep Learning Practitioner**


<a href="https://github.com/rohitkuk">
  <img align="left" alt="Rohit Kukreja Github" width="22px" src="https://cdn-icons-png.flaticon.com/512/25/25231.png" />
</a>
<a href="https://twitter.com/_Rohitkukreja">
  <img align="left" alt="Rohit Kukreja | Twitter" width="22px" src="https://raw.githubusercontent.com/peterthehan/peterthehan/master/assets/twitter.svg" />
</a>
<a href="https://www.linkedin.com/in/rohitkuk/">
  <img align="left" alt="Rohit's's LinkedIN" width="22px" src="https://raw.githubusercontent.com/peterthehan/peterthehan/master/assets/linkedin.svg" />
</a>







#Setup

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%%capture
!git clone https://github.com/dmuinoo/gestual/

# Install YOLOX Dependencies

In [ ]:
%%capture
%cd /content/gestual
!pip3 install -U pip && pip3 install -r requirements.txt
!pip3 install -v -e .  
!pip uninstall -y torch torchvision torchaudio
# May need to change in the future if Colab no longer uses CUDA 11.0
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

#Install Nvidia Apex

In [ ]:
%%capture
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

# Import the Dataset


In [ ]:
%cd /content/gestual/
!curl -L "https://public.roboflow.com/ds/RM02OBPfKD?key=6IpgAWtQ0U" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
root = "/content/gestual/train/"
%mkdir /content/gestual/datasets/VOCdevkit/
!cp -a "/content/gestual/train/." "/content/gestual/datasets/VOCdevkit"

# Prepare dataset as per VOC

In [ ]:
%cd /content/gestual/
%mkdir "/content/gestual/datasets/VOCdevkit/VOC2007"
!python3 voc_txt.py "/content/gestual/datasets/VOCdevkit/"
%mkdir "/content/gestual/datasets/VOCdevkit/VOC2012"
!cp -r "/content/gestual/datasets/VOCdevkit/VOC2007/." "/content/gestual/datasets/VOCdevkit/VOC2012"


#Update the default classes


In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
##REPLACE this cell with your classnames stripped of whitespace and lowercase
%%writetemplate /content/gestual/yolox/data/datasets/voc_classes.py

VOC_CLASSES = (
  'pistol'
)

In [ ]:
##REPLACE this cell with your classnames stripped of whitespace and lowercase
%%writetemplate /content/gestual/yolox/data/datasets/coco_classes.py

COCO_CLASSES = (
  'pistol'
)

In [ ]:
NUM_CLASSES = 1
!sed -i -e 's/self.num_classes = 20/self.num_classes = {NUM_CLASSES}/g' "/content/gestual/exps/example/yolox_voc/yolox_voc_s.py"

#Training

In [ ]:
#Download the weights
%cd /content/gestual
!wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_s.pth

In [ ]:
%env OUT_DIR= /content/gdrive/MyDrive/gestual
!mkdir = /content/gdrive/MyDrive/gestual
!python tools/train.py -f exps/example/yolox_voc/yolox_voc_s.py -d 1 -b 16 --fp16 -o -c /content/gestual/yolox_s.pth

# Inference



In [ ]:
from google.colab import files
files.upload()

In [ ]:
%env OUT_DIR= /content/gdrive/MyDrive/gestual
VIDEO_PATH = "/content/gdrive/MyDrive/yolo/prueba2.mp4"
MODEL_PATH = "/content/gdrive/MyDrive/gestual/YOLOX_outputs/yolox_voc_s/last_epoch_ckpt.pth.tar"
!python tools/demo.py video -f /content/gestual/exps/example/yolox_voc/yolox_voc_s.py -c {MODEL_PATH} --path {VIDEO_PATH} --conf 0.25 --nms 0.45 --save_result --device gpu